In [1]:
import numpy as np  
import numpy.polynomial.legendre as npl

In [45]:
class Cuadrature:
    def __init__(self,nodes=None,weight=None,num_points=None):
        if (nodes!=None and weight!=None):
            self.nodes = nodes
            self.weight = weight
            print(self.nodes,self.weight)
        elif(num_points!=None):
            self.nodes, self.weight= npl.leggauss(num_points)
            print(self.nodes,self.weight)
        else:
            raise Exception("No has metido correctamente los parámetros en la clase")
    def getNodes(self):
        return self.nodes
    def getWeight(self):
        return self.weight
    def Integrate(self,f):
        return self.Integrate_List(f(np.array(self.nodes)))
    def Integrate_List(self,lista):
        if (len(lista)==len(self.weight)):
            return np.array(lista).dot(np.array(self.weight))
        else:
            raise Exception("No has metido correctamente la lista")
        

In [15]:
Cuadrature(4)

Exception: No has metido correctamente los parámetros en la clase

In [46]:
cuad_gauss=Cuadrature(num_points=6)

[-0.93246951 -0.66120939 -0.23861919  0.23861919  0.66120939  0.93246951] [0.17132449 0.36076157 0.46791393 0.46791393 0.36076157 0.17132449]


In [47]:
f= lambda x: x**10
cuad_gauss.Integrate(f)

0.18181818181818124

In [50]:
lista=[2,4,5,0]
cuad_gauss.Integrate_List(lista)

Exception: No has metido correctamente la lista